In [1]:
!pip install pyngrok

In [2]:
!ngrok config add-authtoken YOUR_NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [3]:
!pip install fastapi nest-asyncio uvicorn

In [4]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
from fastapi import HTTPException
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
# Set up Llama model from Hugging Face
from huggingface_hub import login

# Replace with your Hugging Face token
login("HUGGINGFACE_API")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# FastAPI app setup
app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, 
    allow_origins=['*'], 
    allow_credentials=True, 
    allow_methods=['*'], 
    allow_headers=['*'], 
)

In [9]:
# Initialize the model and tokenizer (LLAMA)
class QuestionAnsweringModel:
    def __init__(self):
        self.tokenizer = None
        self.model = None

    def load_model(self):
        """Load LLAMA model and tokenizer"""
        model_name = "meta-llama/Llama-3.2-1B-Instruct"   # Replace with the correct model identifier
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.model.eval()

    def predict(self, question: str, temperature: float = 0.7) -> str:
        """Run the model to answer a question with adjustable temperature"""
        if not self.model:
            raise HTTPException(status_code=400, detail="Model is not loaded")
        
        inputs = self.tokenizer(question, return_tensors="pt")
        
        # Generate text with adjusted temperature
        outputs = self.model.generate(
            inputs["input_ids"], 
            max_length=200, 
            temperature=temperature,  # Adjust temperature here
            top_k=50,  # You can also adjust other parameters like top_k or top_p for more control
            top_p=0.95
        )
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return answer

In [ ]:
# Pydantic model for input and output
class Question(BaseModel):
    question: str

class Answer(BaseModel):
    answer: str

# Load model
qa_model = QuestionAnsweringModel()

# FastAPI routes
@app.get('/')
def index():
    return {'message': 'Welcome to the LLAMA Question Answering API!'}

@app.post('/answer')
async def get_answer(question: Question) -> Answer:
    """
    Example post request body:
    {
        "question": "What is the capital of France?"
    }
    Example response:
    {
        "answer": "The capital of France is Paris."
    }
    """
    answer = qa_model.predict(question.question)
    return Answer(answer=answer)

# Load model asynchronously on startup
@app.on_event("startup")
async def startup():
    qa_model.load_model()

# Set up the FastAPI app to run on a public URL via ngrok
port = 8004
ngrok_tunnel = ngrok.connect(port)
print(f"Public URL: {ngrok_tunnel.public_url}")

nest_asyncio.apply()
uvicorn.run(app, port=port)

/tmp/ipykernel_30/4161319584.py:32: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


Public URL: https://bc10-35-237-44-30.ngrok-free.app


INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8004 (Press CTRL+C to quit)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
